In [5]:
import os
import pandas as pd

In [19]:
base_path  = 'legacy_dataset'
images_path = os.path.join(base_path,'images')
gt_path = os.path.join(base_path,'gt_text.txt')

with open(gt_path, 'r') as f:
    data = f.readlines()
f.close()

file_name_list = [ d.split(' ')[0] for d in data ]
labels_list = [ d.split('.jpg ')[1].strip() for d in data ]

df = pd.DataFrame({'file_name': file_name_list, 'text':labels_list })
df.head()
df.to_csv(os.path.join(base_path,'gt_text.csv'), index=False, index_label=False)

# with open(gt_path, 'w') as f:
#     for line in data:
#         f.write(line)
# f.close()


In [20]:
from tensorflow.keras.layers.experimental.preprocessing import StringLookup
from tensorflow import keras

import matplotlib.pyplot as plt
import tensorflow as tf

In [21]:

def distortion_free_resize(image, img_size):
    w, h = img_size
    image = tf.image.resize(image, size=(h, w), preserve_aspect_ratio=True)

    # Check tha amount of padding needed to be done.
    pad_height = h - tf.shape(image)[0]
    pad_width = w - tf.shape(image)[1]

    # Only necessary if you want to do same amount of padding on both sides.
    if pad_height % 2 != 0:
        height = pad_height // 2
        pad_height_top = height + 1
        pad_height_bottom = height
    else:
        pad_height_top = pad_height_bottom = pad_height // 2

    if pad_width % 2 != 0:
        width = pad_width // 2
        pad_width_left = width + 1
        pad_width_right = width
    else:
        pad_width_left = pad_width_right = pad_width // 2

    image = tf.pad(
        image,
        paddings=[
            [pad_height_top, pad_height_bottom],
            [pad_width_left, pad_width_right],
            [0, 0],
        ],
    )

    image = tf.transpose(image, perm=[1, 0, 2])
    image = tf.image.flip_left_right(image)
    return image


In [22]:
batch_size = 64
padding_token = 99
image_width = 128
image_height = 32


def preprocess_image(image_path, img_size=(image_width, image_height)):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, 1)
    image = distortion_free_resize(image, img_size)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def vectorize_label(label):
    label = char_to_num(tf.strings.unicode_split(label, input_encoding="UTF-8"))
    length = tf.shape(label)[0]
    pad_amount = max_len - length
    label = tf.pad(label, paddings=[[0, pad_amount]], constant_values=padding_token)
    return label


def process_images_labels(image_path, label):
    image = preprocess_image(image_path)
    label = vectorize_label(label)
    return {"image": image, "label": label}


# def prepare_dataset(image_paths, labels):
#     dataset = tf.data.Dataset.from_tensor_slices((image_paths, labels)).map(
#         process_images_labels, num_parallel_calls=AUTOTUNE
#     )
#     return dataset.batch(batch_size).cache().prefetch(AUTOTUNE)


In [26]:
%load_ext tensorboard


The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [27]:
%tensorboard --logdir logs/train_data